In [6]:
import requests
import json
import time

API_KEY = '30OOGV78CCLXQM2P'
STOCK_SYMBOLS = ['IBM', 'AAPL', 'MSFT']
FUNCTION = 'TIME_SERIES_INTRADAY'
INTERVAL = '5min'

all_data = {}

for symbol in STOCK_SYMBOLS:
    url = f'https://www.alphavantage.co/query?function={FUNCTION}&symbol={symbol}&interval={INTERVAL}&apikey={API_KEY}'
    response = requests.get(url)

    if response.status_code == 200:
        all_data[symbol] = response.json()
        print(f"Data retrieved for {symbol}")
    else:
        print(f"Failed to retrieve data for {symbol}: {response.status_code}")

    time.sleep(12)

file_name = 'all_stocks_data.json'
with open(file_name, 'w') as file:
    json.dump(all_data, file, indent=4)


Data retrieved for IBM
Data retrieved for AAPL
Data retrieved for MSFT


In [1]:
import pandas as pd
json_file_path = 'all_stocks_data.json'
with open(json_file_path, 'r') as file:
    json_data = pd.read_json(file)

json_data.head()

,IBM,AAPL,MSFT
Meta Data,"{'1. Information': 'Intraday (5min) open, high...","{'1. Information': 'Intraday (5min) open, high...","{'1. Information': 'Intraday (5min) open, high..."
Time Series (5min),{'2023-11-24 17:00:00': {'1. open': '155.1800'...,{'2023-11-24 16:55:00': {'1. open': '189.9350'...,{'2023-11-24 16:55:00': {'1. open': '377.3530'...


In [11]:
def extract_time_series(data, stock_symbol):
    time_series = data[stock_symbol]['Time Series (5min)']
    df = pd.DataFrame(time_series).T
    df['Stock Symbol'] = stock_symbol
    return df

ibm_df = extract_time_series(json_data, 'IBM')
aapl_df = extract_time_series(json_data, 'AAPL')
msft_df = extract_time_series(json_data, 'MSFT')

combined_df = pd.concat([ibm_df, aapl_df, msft_df]).reset_index()
combined_df.rename(columns={'index': 'Timestamp'}, inplace=True)

csv_output_path = 'data.csv'
combined_df.to_csv(csv_output_path, index=False)

csv_output_path

'data.csv'

In [1]:
import os
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "data-management-406306-d02f9bb50c44.json"


In [2]:
client = bigquery.Client()

project_id = "data-management-406306"

In [5]:
dataset_id = "data-management-406306.Stocks_data"
table_id = "data-management-406306.Stocks_data.Stockstable"  

In [6]:
source_file = "stocks_d.json"

table_ref = client.dataset(dataset_id).table(table_id)

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
)

with open(source_file, "rb") as source_file:
    load_job = client.load_table_from_file(
        source_file,
        table_ref,
        location="US", 
        job_config=job_config,
    )

load_job.result()

print(f"Loaded {load_job.output_rows} rows into {table_id}")

NotFound: 404 POST https://bigquery.googleapis.com/upload/bigquery/v2/projects/data-management-406306/jobs?uploadType=resumable: Not found: Dataset data-management-406306:data-management-406306.Stocks_data

In [18]:
df = pd.read_csv('data.csv')
df.head()

,Timestamp,1. open,2. high,3. low,4. close,5. volume,Stock Symbol
0,2023-11-24 17:00:00,155.18,155.18,155.18,155.18,335049,IBM
1,2023-11-24 16:55:00,155.11,155.18,155.11,155.18,40,IBM
2,2023-11-24 16:50:00,155.10,155.10,155.10,155.10,1,IBM
3,2023-11-24 16:40:00,155.05,155.05,155.05,155.05,40,IBM
4,2023-11-24 16:35:00,155.15,155.15,155.15,155.15,100,IBM


In [21]:
df.shape

(300, 7)

In [15]:
df = df.dropna(subset=['2. high'])

In [20]:
df.columns = ['Timestamp', 'open', 'high', 'low', 'close', 'volume', 'Stock Symbol']
df.head()


,Timestamp,open,high,low,close,volume,Stock Symbol
0,2023-11-24 17:00:00,155.18,155.18,155.18,155.18,335049,IBM
1,2023-11-24 16:55:00,155.11,155.18,155.11,155.18,40,IBM
2,2023-11-24 16:50:00,155.10,155.10,155.10,155.10,1,IBM
3,2023-11-24 16:40:00,155.05,155.05,155.05,155.05,40,IBM
4,2023-11-24 16:35:00,155.15,155.15,155.15,155.15,100,IBM
